In [1]:
import json
import time 

from kafka import KafkaProducer

In [56]:
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

In [55]:
producer.bootstrap_connected()

AttributeError: 'KafkaProducer' object has no attribute 'bootstrap_connected'

In [6]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
took 0.50 seconds


In [44]:
#we do not need a spark session to create the dataframe as we are no longer inferring the schema
#we want to leave all attributes as string
import pyspark
from pyspark.sql import SparkSession

In [45]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/17 16:58:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [28]:
df = spark.read \
    .option("header", "true") \
    .csv('green_tripdata_2019-10.csv')

In [ ]:
.option("inferSchema", "true") \

In [29]:
df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- trip_type: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)



In [30]:
df_green = df \
    .select(['lpep_pickup_datetime','lpep_dropoff_datetime','PULocationID','DOLocationID','passenger_count','trip_distance','tip_amount'])

In [31]:
df_green.show()

+--------------------+---------------------+------------+------------+---------------+-------------+----------+
|lpep_pickup_datetime|lpep_dropoff_datetime|PULocationID|DOLocationID|passenger_count|trip_distance|tip_amount|
+--------------------+---------------------+------------+------------+---------------+-------------+----------+
| 2019-10-01 00:26:02|  2019-10-01 00:39:58|         112|         196|              1|         5.88|         0|
| 2019-10-01 00:18:11|  2019-10-01 00:22:38|          43|         263|              1|          .80|         0|
| 2019-10-01 00:09:31|  2019-10-01 00:24:47|         255|         228|              2|         7.50|         0|
| 2019-10-01 00:37:40|  2019-10-01 00:41:49|         181|         181|              1|          .90|         0|
| 2019-10-01 00:08:13|  2019-10-01 00:17:56|          97|         188|              1|         2.52|      2.26|
| 2019-10-01 00:35:01|  2019-10-01 00:43:40|          65|          49|              1|         1.47|    

In [32]:
import pandas as pd

In [33]:
pandas_green_df = df_green.toPandas()

In [34]:
for row in pandas_green_df.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    print(row_dict)
    break

{'lpep_pickup_datetime': '2019-10-01 00:26:02', 'lpep_dropoff_datetime': '2019-10-01 00:39:58', 'PULocationID': '112', 'DOLocationID': '196', 'passenger_count': '1', 'trip_distance': '5.88', 'tip_amount': '0'}


In [39]:
topic_name = 'test-topic'

for row in pandas_green_df.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    print(row_dict)
    break
    
producer.send(topic_name, value=row_dict)
producer.flush()

{'lpep_pickup_datetime': '2019-10-01 00:26:02', 'lpep_dropoff_datetime': '2019-10-01 00:39:58', 'PULocationID': '112', 'DOLocationID': '196', 'passenger_count': '1', 'trip_distance': '5.88', 'tip_amount': '0'}


In [43]:
t0 = time.time()

topic_name = 'green-trips'

for row in pandas_green_df.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    producer.send(topic_name, value=row_dict)

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

took 67.31 seconds


In [46]:
spark.stop()

In [62]:
import pyspark
from pyspark.sql import SparkSession

In [63]:
pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

In [69]:
kafka_jar_package = 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1242389542305'

In [70]:
pyspark.__version__

'3.3.2'

In [71]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

24/03/17 17:25:36 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [68]:
# now we connect to the stream
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

AnalysisException:  Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".        